In [55]:
# 基本モジュールのインポート
import sys, requests, datetime, lxml, os
from datetime import timedelta

# 日付を簡易的に変更するモジュール
# https://qiita.com/xza/items/9618e25a8cb08c44cdb0
from dateutil.relativedelta import relativedelta

# ウェブスクレイピング用モジュール
# https://qiita.com/itkr/items/513318a9b5b92bd56185
from lxml.html import fromstring
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# 並列処理を行う上で必要なモジュール
# https://docs.python.org/ja/3/library/multiprocessing.html
from  multiprocessing import Process

# データフレーム格納処理に使用するモジュール
import numpy as np
import pandas as pd

# 個人で作成したスクリプト：関数やコードのリストなど
sys.path.insert(0, './src/')
out_dir = './data/h_desc/'

# ディレクトリの定義
# ディレクトリ作成ユーティリティ
h_desc_dir = './data/h_desc/'
r_res_dir = './data/r_res/'

In [56]:
!date

Tue Jun 30 16:25:22 JST 2020


### 1. 地方競馬場コードの定義

データ取得先：**'https://www.jbis.or.jp/'**

地方競馬場のレース結果と出場競走馬を取得するため、上記ウェブサイト上で定義されている各競馬場のコードを定義する。


In [7]:
chihou_keibajo = {
    "岩見沢": '205', "帯広": '206', "旭川": '207', 
    "札幌": '208', "函館": '209', "門別": '236', 
    "水沢": '211', "盛岡": '210', "上山": '212', 
    "新潟": '213', "三条": '214', "足利": '215',
    "宇都宮": '216', "高崎": '217', "浦和": '218',
    "船橋": '219', "大井": '220', "川崎": '221',
    "金沢": '222', "笠松": '223', "名古屋": '224',
    "中京": '225', "園田": '227', "姫路": '228',
    "益田": '229', "福山": '230', "高知": '231',
    "佐賀": '232', "荒尾": '233', "中津": '234' 
}

### 2. スクレイピング

In [20]:
# seleniumではなくbeautirfulsoupを代わりに使用して今回はデータ取得する

# optionを定義する。より早くスクレイピングを行うため、
# ヘッドレスモード、window-sizeは1080pでchromeを起動する。
# options = Options()
# options.headless = True
# options.add_argument("--window-size=1920,1200")
# DRIVER_PATH = './chromedriver/chromedriver'
# driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

# seleniumを使用するうえでマルチプロセスでデータ取得するための関数
# def run(options, executable_path, url, directory):
#     driver = webdriver.Chrome(options=options, executable_path=executable_path)
#     driver.get(url)
#     f_name = driver.find_element_by_class_name("hdg-l1-01")
#     pd.read_html(driver.page_source)[0].to_csv(directory + '/' + f_name.text + '.csv', encoding='utf-8')
#     driver.close()

In [86]:
# データ抽出開始年月
date_counter = datetime.date(2019, 10, 1)

# データ抽出先ベースURL
baseURL = 'https://www.jbis.or.jp'

In [87]:
# セッションの新スタンス化
s = requests.Session()

# URL先のページソースの取得
r_calendar = s.get(baseURL + '/race/calendar/' + date_counter.strftime('%Y%m') + '/')

# 取得したページソースをxml形式に変換する
soup_calendar = bs(r.content, 'lxml')

In [89]:
# ページソースから日付ごとに各競技場のレース結果のURLを取得する
hrefs = [baseURL + a['href'] for a in soup.find("tbody").find_all("a", href=True)]
hrefs

['https://www.jbis.or.jp/race/calendar/20191001/220/',
 'https://www.jbis.or.jp/race/calendar/20191001/222/',
 'https://www.jbis.or.jp/race/calendar/20191001/224/',
 'https://www.jbis.or.jp/race/calendar/20191001/236/',
 'https://www.jbis.or.jp/race/calendar/20191002/220/',
 'https://www.jbis.or.jp/race/calendar/20191002/224/',
 'https://www.jbis.or.jp/race/calendar/20191002/227/',
 'https://www.jbis.or.jp/race/calendar/20191002/236/',
 'https://www.jbis.or.jp/race/calendar/20191003/220/',
 'https://www.jbis.or.jp/race/calendar/20191003/224/',
 'https://www.jbis.or.jp/race/calendar/20191003/227/',
 'https://www.jbis.or.jp/race/calendar/20191003/236/',
 'https://www.jbis.or.jp/race/calendar/20191004/220/',
 'https://www.jbis.or.jp/race/calendar/20191004/224/',
 'https://www.jbis.or.jp/race/calendar/20191004/227/',
 'https://www.jbis.or.jp/race/calendar/20191005/104/',
 'https://www.jbis.or.jp/race/calendar/20191005/105/',
 'https://www.jbis.or.jp/race/calendar/20191005/108/',
 'https://

In [194]:
# 飛んだ先のURLからテーブルを取得する
r_race_res = s.get(hrefs[0])
soup_r_race_res = bs(r_race_res.content)
table = soup_r_race_res.find('table')
pd.read_html(str(table))[0]

,R,レース名,距離,頭数,勝馬 生産牧場,勝馬 生産牧場.1,人気,２着馬 生産牧場,２着馬 生産牧場.1,人気.1
0,1,サラ系３歳,ダ1600m,8,5,マイネルオルガノ 松本牧場,1,6,カズノマックイーン 新冠橋本牧場,4
1,2,サラ系３歳,ダ1400m,14,5,トラサン 三石ファーム,5,4,バニラオーキッド スピードフアーム,2
2,3,サラ系３歳,ダ1200m,14,6,ピンクレイク 小倉 光博,2,11,ラブアバンティ 坂戸 節子,5
3,4,サラ系２歳,ダ1600m,7,5,ファンシーアップ 追分ファーム,1,6,ロイヤルペガサス 槙本牧場,2
4,5,サラ系一般 Ｃ３三四,ダ1600m,10,5,ストームフレイバー 新冠橋本牧場,1,2,アキラノテソーロ 丸幸小林牧場,7
5,6,サラ系一般 Ｃ３三四,ダ1200m,13,11,ジェットバローズ 川島牧場,8,8,エンジェルサイン ノーザンファーム,3
6,7,サラ系一般 Ｃ１七八,ダ1600m,10,8,マイネルアーリー 武田牧場,2,3,イーストコースト ナカノファーム,1
7,8,サラ系一般 Ｃ１七八,ダ1200m,14,4,イオラナ 千葉飯田牧場,1,3,ヤマクジラ 坂本牧場,2
8,9,秋風賞 Ｂ２五Ｂ３五,ダ1000m,9,3,パルドン 武田牧場,1,9,ミントフレイバー 村田牧場,4
9,10,とき特別,ダ1200m,8,2,プリマガリーナ 伏木田牧場,1,5,ジャギーチェーン 山田牧場,5


In [128]:
# レース結果詳細のURLを取得する
race_detail_href =[a['href'] for a in table.find_all("a", href=True) if a['href'].find('race') != -1]
race_detail_href

['/race/result/20191001/220/01/',
 '/race/result/20191001/220/02/',
 '/race/result/20191001/220/03/',
 '/race/result/20191001/220/04/',
 '/race/result/20191001/220/05/',
 '/race/result/20191001/220/06/',
 '/race/result/20191001/220/07/',
 '/race/result/20191001/220/08/',
 '/race/result/20191001/220/09/',
 '/race/result/20191001/220/10/',
 '/race/result/20191001/220/11/',
 '/race/result/20191001/220/12/']

In [139]:
# レース詳細結果に飛び、レース結果を取得する
r_race_detail = s.get(baseURL + race_detail_href[0])
soup_r_race_detail = bs(r_race_detail.content, 'lxml')
r_tables = soup_r_race_detail.find_all('table')
r_table_hrefs =[a['href'] for a in r_tables[1].find_all("a", href=True) if a['href'].find('horse') != -1]
r_table_hrefs

['/horse/0001216669/',
 '/horse/0000294840/',
 '/horse/0000804200/',
 '/horse/0001219790/',
 '/horse/0001110554/',
 '/horse/0001090845/',
 '/horse/0001215898/',
 '/horse/0001163245/',
 '/horse/0000754075/',
 '/horse/0001222525/',
 '/horse/0000848478/',
 '/horse/0001135225/',
 '/horse/0001222802/',
 '/horse/0000905163/',
 '/horse/0001109187/',
 '/horse/0001214028/',
 '/horse/0001111834/',
 '/horse/0000324970/',
 '/horse/0001215452/',
 '/horse/0001028940/',
 '/horse/0000734888/',
 '/horse/0001216146/',
 '/horse/0000991842/',
 '/horse/0000760864/']

In [154]:
# 各馬ごとに基本情報を取得する
r_desc = s.get(baseURL + r_table_hrefs[0])
soup_r_desc = bs(r_desc.content)
r_desc_table = soup_r_desc.find_all('table')
r_desc_table[0]

<table border="1" cellspacing="0" class="tbl-data-05 reset-mb-40">
<tbody>
<tr>
<th class="w15">登録</th>
<td>地方現役</td>
<th class="cell-bl-01 w15">品種</th>
<td>サラ</td>
</tr>
<tr class="even">
<th>性別</th>
<td>  牡</td>
<th class="cell-bl-01">生年月日</th>
<td>
<ul class="list-inline-01 reset-mb-00">
<li class="first-child">2016/03/11</li>
<li>4歳</li>
</ul>
</td>
</tr>
<tr>
<th>馬主</th>
<td><a href="/horse/owner/0120000043/">谷謙介</a></td>
<th class="cell-bl-01">毛色</th>
<td>鹿毛</td>
</tr>
<tr class="even">
<th>調教師</th>
<td><a href="/horse/trainer/011043/">大島　静夫</a>（佐　賀）</td>
<th class="cell-bl-01">生産牧場</th>
<td><a href="/breeder/0000000592/">松本牧場</a></td>
</tr>
<tr>
<th>産地</th>
<td>三石産</td>
<th class="cell-bl-01">戦績</th>
<td>
<ul class="list-inline-01 reset-mb-00">
<li class="first-child">国内：23戦6勝</li>
</ul>
</td>
</tr>
<tr class="even">
<th>総賞金</th>
<td colspan="3">446.6万円</td>
</tr>
</tbody>
</table>

In [193]:
# 競走馬情報のテーブルを作成する
cols = ["番号", '競走馬名']
data = []

# 番号をURLから取得
data.append(r_table_hrefs[0].split('/')[-2])

# 競走馬名を取得
data.append(soup_r_desc.find('h1', {'class':'hdg-l1-02'}).text)

# 競走馬の属性を取得
for th, td in zip(r_desc_table[0].find_all('th'), r_desc_table[0].find_all('td')):
    cols.append(th.text)
    data.append(td.text)

# 競走馬の成績を取得
tot_res = pd.read_html(str(r_desc_table))[2]
tot_res.rename({'Unnamed: 0': '種類'}, axis=1, inplace=True)


f_


,番号,競走馬名,登録,品種,性別,生年月日,馬主,毛色,調教師,生産牧場,...,8,9,10,11,12,13,14,15,16,17
0,0001216669,マイネルオルガノ,地方現役,サラ,牡,\n\n2016/03/11\n4歳\n\n,谷謙介,鹿毛,大島 静夫（佐 賀）,松本牧場,...,SSI = 0.00,地方,合計,20,6,2,3,9,446.6,SSI = 1.00


In [ ]:
while not date_counter.strftime("%Y%m") == "202007":
    
    # 月で試合結果を抽出する
    ym = date_counter.strftime("%Y%m")
    #print(ym)
    driver.get(url + ym + '/')

    # 抽出開始
    # ボディ
    tbl = driver.find_element_by_class_name("tbl-calendar-01")

    # テーブル列
    tbl_rows = tbl.find_elements_by_xpath(".//tbody/tr")

    for tbl_row in tbl_rows:
    
        # セル列
        try:
            cells = tbl_row.find_elements_by_xpath(".//td")
        except:
            #print("td failed.")
            continue
        
        for cell in cells:

            # リンク先を取得
            try: 
                races = cell.find_elements_by_xpath(".//ul/li")
            except:
                #print(".//ul/li failed.")
                continue
                
            for race in races:
                try:
                    race = race.find_element_by_tag_name("a")
                    race_href = race.get_attribute('href')
                    race_loc = race.find_element_by_tag_name('img').get_attribute('alt')
                    #print("through here.")
                    # 地方競馬場でない場合、次のループへ
                    if chihou_keibajo[race_loc] is None:
                        print("not chihou keiba skipping...")
                        continue
    
                    # 地方競馬場のコードで出力ディレクトリを作成
                    if not os.path.isdir(out_r_res + chihou_keibajo[race_loc]):
                        os.mkdir(out_r_res + chihou_keibajo[race_loc])
    
                    directory = out_r_res + chihou_keibajo[race_loc] + '/' + race_href.split('/')[-3][:-2]
                    #print(race_href.split('/')[-3])
                    # 日付ごとにディレクトリを作成する
                    if not os.path.isdir(directory):
                        os.mkdir(directory)
                        
                        
                    # プロセス
                    p = Process(target=run, args=(options, DRIVER_PATH, race_href, directory))
                    p.start()
                    p.join()
                    p.close()
                except:
                    #print("empty cell...")
                    #print(".//a failed.")
                    continue
    # 1カ月先に進める
    date_counter = date_counter + relativedelta(months=1)

Process Process-585:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-22-c0044efa5850>", line 3, in run
    driver.get(url)
  File "/home/masa/.local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "/home/masa/.local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/home/masa/.local/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: timeout: Timed out receiving message from renderer: -0.015
  (Session info: headless chrome=83.0.41

In [197]:
pd.read_csv('./data/h_desc/20200601_210_01.csv')

,番号,競走馬,登録,品種,性別,生年月日,馬主,毛色,調教師,生産牧場,...,11,12,13,14,15,16,17,繋養先,種付料,写真
0,1187584,バトルクウ,地方現役,サラ,牝,\n\n2014/03/27\n6歳\n\n,大田恭充,青鹿毛,橘 友和（盛 岡）,トウショウ産業株式会社トウショウ牧場,...,69.0,6.0,10.0,9.0,44.0,244.3,SSI = 0.31,NaN,NaN,NaN
1,714411,ケイムホーム(USA),繁殖,サラ,牡,\n\n1999/03/29\n21歳\n\n,,黒鹿毛,（海 外）,John A. Toffan & Trudy McCaffery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n\n（公社）日本軽種馬協会九州種馬場 鹿児島県\n電話：0994-78-3011 \n\n,\n\n2020年度：受胎確認後10万円(流死産時又は産駒死亡時に返還)\n\n,\n\n\n
2,798300,シリウストウショウ,繁殖,サラ,牝,\n\n2004/05/13\n16歳\n\n,トウショウ産業（株）,栗毛,鶴留 明雄（栗 東）,トウショウ産業株式会社トウショウ牧場,...,1.0,0.0,0.0,0.0,1.0,0.0,SSI = 0.00,NaN,NaN,NaN
3,1222888,リュウグウオー,地方現役,サラ,牡,\n\n2016/04/10\n4歳\n\n,深澤朝房,鹿毛,瀬戸 幸一（水 沢）,シンボリ牧場,...,32.0,1.0,5.0,4.0,22.0,90.2,SSI = 0.13,NaN,NaN,NaN
4,1090360,オルフェーヴル,繁殖,サラ,牡,\n\n2008/05/14\n12歳\n\n,（有）サンデーレーシング,栗毛,池江 泰寿（栗 東）,（有）社台コーポレーション白老ファーム,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n\n社台スタリオンステーション 勇払郡 安平町\n電話：0145-22-4581 \n\n,\n\n2020年度：受胎確認後300万円(フリーリターン特約付)\n\n,\n\n\n\n\n\n\n\n\n\n\n
5,1132997,グッドカルマ(ITY),繁殖,サラ,牝,\n\n2008/02/27\n12歳\n\n,,鹿毛,（海 外）,Soc Ag Al Deni SRL,...,11.0,6.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
6,1201109,コスモミクニグロ,地方現役,サラ,牡,\n\n2015/02/22\n5歳\n\n,谷謙介,鹿毛,佐々木 由則（水 沢）,ビッグレッドファーム,...,55.0,8.0,8.0,7.0,32.0,252.4,SSI = 0.21,NaN,NaN,NaN
7,762445,マツリダゴッホ,繁殖,サラ,牡,\n\n2003/03/15\n17歳\n\n,高橋文枝,鹿毛,国枝 栄（美 浦）,岡田スタツド,...,1.0,0.0,0.0,0.0,1.0,NaN,NaN,\n\nレツクススタツド 日高郡 新ひだか\n電話：0146-42-3600 \n\n,\n\n2020年度：受胎確認後50万円(フリーリターン特約付)\n産駒誕生後80万円\n,\n\n
8,450402,グリントインハーアイ(USA),繁殖,サラ,牝,1996/03/09,,鹿毛,（海 外）,Katom Ltd & Barronstown Stud,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1220557,ストーリーズ,地方現役,サラ,牝,\n\n2016/04/17\n4歳\n\n,石川諭,鹿毛,佐藤 雅彦（水 沢）,服部 牧場,...,10.0,3.0,0.0,0.0,7.0,109.7,SSI = 0.89,NaN,NaN,NaN
